In [1]:
import torch
from PIL import Image
import torchvision
from torch.fx.experimental.partitioner_utils import Device
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import Conv2d,MaxPool2d, Flatten, Linear
from torch.utils.tensorboard import SummaryWriter
import time

In [2]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=nn.Sequential(
            Conv2d(in_channels=3,out_channels=32,kernel_size=5,padding=2),
            MaxPool2d(kernel_size=2),
            Conv2d(in_channels=32,out_channels=32,kernel_size=5,padding=2),
            MaxPool2d(kernel_size=2),
            Conv2d(in_channels=32,out_channels=64,kernel_size=5,padding=2),
            MaxPool2d(kernel_size=2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10)
        )
    def forward(self, x):
        x=self.model(x)
        return x

In [7]:
# 加载并处理图片
image_pth="dataset/dog.png"
img=Image.open(image_pth)
print(img.size)
transform_compose=torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(32,32)),
    torchvision.transforms.ToTensor()
])
img=transform_compose(img)
img=img.unsqueeze(0)
print(img.shape)


(742, 1024)
torch.Size([1, 3, 32, 32])


In [8]:
#加载模型
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=torch.load("./model/cifar/cifar10_model_20.pth",weights_only= False)
model.eval()
model.to(device)
img=img.to(device)
with torch.no_grad():
    output=model(img)
    print(output.argmax(1))

tensor([3], device='cuda:0')


In [11]:
#加载模型
model=torch.load("./model/cifar/cifar10_model_20.pth",weights_only= False,map_location=torch.device("cpu"))
model.eval()
img=img.to(torch.device("cpu"))
with torch.no_grad():
    output=model(img)
    print(output.argmax(1))

tensor([3])
